In [ ]:
import numpy as np
from coverage import *

from shapely.geometry import Point
import geopandas as gpd
import pandas as pd
import branca
import folium


In [ ]:
num_days = 0.25

# Pull TLEs for SPG4 propagation
tles = gen_sats(sat_nos=[39084,49260])

# Generate time range
# times = gen_times(start_yr=2022, start_mo=6, start_day=10, days=num_days, step_min=1)
times = gen_times(start_yr=2022, start_mo=6, start_day=15, days=num_days, step_min=0.22) # Landsat duty cycle ~23 sec

In [ ]:
cmap = {'LANDSAT 9': '#0000ffff', 'LANDSAT 8': '#ff0000ff'}

In [ ]:
# Calculate all sub-satellite points (SSPs) for sats/ times
ssps = []
for tle in tles:
    sat = tle[0]
    for time in times:
        ssp_lat, ssp_lon, d = get_los(sat, time)

        xyz_dist_rates = sat.at(time).frame_xyz_and_velocity(itrs)
        # xyz_dist = xyz_dist_rates[0]
        z_rate = xyz_dist_rates[1]

        # Descending only filter:
        if z_rate.km_per_s[2] < 0:
            orbit = "descending"
        if z_rate.km_per_s[2] > 0:
            orbit = "ascending"

        ssps.append(
            {
            'time' : time.utc_datetime(),
            'geometry' : Point(ssp_lon.degrees, ssp_lat.degrees),
            'range' : d,
            'color' : cmap[sat.name],
            'orbit' : orbit
            }
        )

In [ ]:
# create geo df of SSPs
ssp_df = gpd.GeoDataFrame(ssps, crs="epsg:4326")

# drop time column and plot
ssp_df.drop("time", axis=1).explore(
    color="color", tooltip=True, style_kwds={"stroke":False}
)

In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
# world.explore()

In [ ]:
# filter for SSPs over land
land_ssp_df = gpd.sjoin(ssp_df, world, predicate='within', how='inner')

# drop time column, filter for descending orbits, and plot
land_ssp_df[land_ssp_df.orbit == "descending"].drop(
    'time', axis=1
    ).explore(
        color="color", tooltip=False, style_kwds={"stroke":False}
        )